## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### OpenAPI
- 한국관광공사 관광빅데이터 정보서비 GW
  - 기본URL : http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList
  - 서비스키 : qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D
  - 피라미터(?뒤)
    - serviceKey = 본인서비스키
    - numOfRows = 한페이지나타낼 결과수(10기본 60으로 하면 하루치 전부다 가져올 수 있음)
    - pageNo = 페이지번호(1)
    - MobileOs = IOS, AND, WIN, ETC(기타)
    - MobileApp = AppTest(테스트)
    - startYmd = 시작일자
    - endYmd = 종료일자

In [24]:
from urllib.request import Request,urlopen
from datetime import datetime
import ssl
import json

In [3]:
def getRequestUrl(url):
    req = Request(url)
    try:
        res = urlopen(req)
        if res.getcode() == 200: # 웹URL 요철결과 ok
            print(f'[{datetime.now()}] Url 요청 성공!')
            return res.read().decode('utf-8')   # 웹으로 전달될때는 URL, utf-8 인코딩, 돌려받을떄는 utf-8 디코딩
    except Exception as e:
        print(f'[{datetime.now()}]getRequestUrl() 호출 예외 : {e}')
        return None

In [22]:
# 파라미터를 변경하면서 OpenAPI 요청할 URL 만들어주는 함수
# 만약 시작일자, 종료일자가 다르면 파라미터 두개 받아야함
serviceKey = 'qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D'
def getTouristDataService(yyyymmdd):
    serviceUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList'
    parameters = f'?serviceKey={serviceKey}'+ \
                 f'&numOfRows = 60' + \
                 '&pageNo = 1' + \
                 'MobileOS=ETC' + \
                 '&MobileApp = AppTest' + \
                 f'&startYmd = {yyyymmdd}' + \
                 f'&endYmd = {yyyymmdd}'
    url = serviceUrl + parameters
    print(url)
    ssl._create_default_https_context = ssl._create_unverified_context
    response = getRequestUrl(url) # OpenAPI URL로 실제 요청

    if response == None:
        return None
    else:
        return response

In [23]:
getTouristDataService('20231201')

http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?serviceKey=qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D&numOfRows = 60&pageNo = 1MobileOS=ETC&MobileApp = AppTest&startYmd = 20231201&endYmd = 20231201
[2024-03-20 12:19:33.410338]getRequestUrl() 호출 예외 : URL can't contain control characters. '/B551011/DataLabService/metcoRegnVisitrDDList?serviceKey=qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D&numOfRows = 60&pageNo = 1MobileOS=ETC&MobileApp = AppTest&startYmd = 20231201&endYmd = 20231201' (found at least ' ')
